# 传统方法消融实验

本notebook进行传统方法的扩展实验，包括：

1. **特征组合消融**：HOG单独 vs. HOG+颜色直方图 vs. HOG+LBP vs. 全部组合
2. **分类器对比**：SVM vs. Logistic Regression vs. MLP
3. **降维策略**：PCA降维 vs. 不降维
4. **样本不平衡处理**：类别权重 vs. 无权重

**注意**：这些实验可能需要较长时间运行，建议在完成E1基线后再进行。


In [ ]:
import sys
from pathlib import Path

REPO_ROOT = Path.cwd()
if REPO_ROOT.name == "notebooks":
    REPO_ROOT = REPO_ROOT.parent

SRC_DIR = (REPO_ROOT / "src").resolve()
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from data.traditional_loader import load_all_splits
from models.traditional_features import (
    extract_hog_features,
    extract_color_histogram,
    extract_lbp_features,
    combine_features,
)
from models.traditional_classifiers import (
    train_svm,
    train_logistic_regression,
    train_mlp,
    evaluate_model,
)
from eval.traditional_eval import (
    plot_confusion_matrix,
    plot_feature_comparison,
    plot_classifier_comparison,
)

sns.set_style("whitegrid")
plt.rcParams["font.sans-serif"] = ["SimHei", "Arial Unicode MS", "DejaVu Sans"]
plt.rcParams["axes.unicode_minus"] = False

# Paths
DATA_DIR = REPO_ROOT / "data"
SPLIT_JSON = DATA_DIR / "splits" / "plantdoc_split_seed42.json"
PROCESSED_ROOT = DATA_DIR / "processed" / "plantdoc_224"
OUTPUTS_DIR = REPO_ROOT / "outputs"
FIGURES_DIR = OUTPUTS_DIR / "figures"
LOGS_DIR = OUTPUTS_DIR / "logs"

print("✓ Setup complete")


✓ Setup complete


## 1. 加载数据并提取所有特征类型

首先提取所有需要的特征类型，然后进行组合实验。


In [ ]:
# Load data
splits_data, class_names = load_all_splits(
    split_json_path=SPLIT_JSON,
    processed_root=PROCESSED_ROOT,
    target_size=(256, 256),
    grayscale=False,
)

X_train, y_train, _ = splits_data["train"]
X_val, y_val, _ = splits_data["val"]
X_test, y_test, _ = splits_data["test"]

print(f"Data loaded: Train={X_train.shape}, Val={X_val.shape}, Test={X_test.shape}")


Loading test: 100%|██████████| 398/398 [00:00<00:00, 430.11it/s]

Data loaded: Train=(1730, 256, 256, 3), Val=(360, 256, 256, 3), Test=(398, 256, 256, 3)


In [ ]:
# Extract all feature types
print("Extracting HOG features...")
X_train_hog = extract_hog_features(X_train)
X_val_hog = extract_hog_features(X_val)
X_test_hog = extract_hog_features(X_test)

print("Extracting color histogram features...")
X_train_color = extract_color_histogram(X_train, bins=32)
X_val_color = extract_color_histogram(X_val, bins=32)
X_test_color = extract_color_histogram(X_test, bins=32)

print("Extracting LBP features...")
X_train_lbp = extract_lbp_features(X_train)
X_val_lbp = extract_lbp_features(X_val)
X_test_lbp = extract_lbp_features(X_test)

print(f"\nFeature dimensions:")
print(f"  HOG: {X_train_hog.shape[1]}")
print(f"  Color: {X_train_color.shape[1]}")
print(f"  LBP: {X_train_lbp.shape[1]}")


Extracting HOG features...
Extracting color histogram features...
Extracting LBP features...

Feature dimensions:
  HOG: 34596
  Color: 96
  LBP: 26


## 2. 特征组合消融实验

对比不同特征组合的效果。


In [ ]:
# Define feature combinations
feature_combinations = {
    "HOG only": {
        "train": X_train_hog,
        "val": X_val_hog,
        "test": X_test_hog,
    },
    "HOG + Color": {
        "train": combine_features({"hog": X_train_hog, "color": X_train_color}, normalize=True)[0],
        "val": combine_features({"hog": X_val_hog, "color": X_val_color}, normalize=True)[0],
        "test": combine_features({"hog": X_test_hog, "color": X_test_color}, normalize=True)[0],
    },
    "HOG + LBP": {
        "train": combine_features({"hog": X_train_hog, "lbp": X_train_lbp}, normalize=True)[0],
        "val": combine_features({"hog": X_val_hog, "lbp": X_val_lbp}, normalize=True)[0],
        "test": combine_features({"hog": X_test_hog, "lbp": X_test_lbp}, normalize=True)[0],
    },
    "HOG + Color + LBP": {
        "train": combine_features({"hog": X_train_hog, "color": X_train_color, "lbp": X_train_lbp}, normalize=True)[0],
        "val": combine_features({"hog": X_val_hog, "color": X_val_color, "lbp": X_val_lbp}, normalize=True)[0],
        "test": combine_features({"hog": X_test_hog, "color": X_test_color, "lbp": X_test_lbp}, normalize=True)[0],
    },
}

print("Feature combinations prepared")


Feature combinations prepared


In [ ]:
# Test each feature combination with SVM
feature_results = []

for feat_name, feat_data in feature_combinations.items():
    print(f"\n{'='*60}")
    print(f"Testing: {feat_name}")
    print(f"{'='*60}")
    
    X_train_feat = feat_data["train"]
    X_val_feat = feat_data["val"]
    X_test_feat = feat_data["test"]
    
    # Train SVM
    model = train_svm(X_train_feat, y_train, kernel="rbf", C=1.0, class_weight="balanced")
    
    # Evaluate
    val_metrics = evaluate_model(model, X_val_feat, y_val, class_names, verbose=False)
    test_metrics = evaluate_model(model, X_test_feat, y_test, class_names, verbose=False)
    
    feature_results.append({
        "feature": feat_name,
        "val_accuracy": val_metrics["accuracy"],
        "val_f1": val_metrics["macro_f1"],
        "test_accuracy": test_metrics["accuracy"],
        "test_f1": test_metrics["macro_f1"],
    })
    
    print(f"Val Accuracy: {val_metrics['accuracy']:.4f}, Val F1: {val_metrics['macro_f1']:.4f}")
    print(f"Test Accuracy: {test_metrics['accuracy']:.4f}, Test F1: {test_metrics['macro_f1']:.4f}")

# Create results DataFrame
feature_df = pd.DataFrame(feature_results)
print("\n" + "="*60)
print("Feature Combination Results Summary")
print("="*60)
print(feature_df.to_string(index=False))



Testing: HOG only


In [ ]:
# Visualize feature comparison
plot_feature_comparison(
    feature_df["feature"].tolist(),
    feature_df["test_accuracy"].tolist(),
    save_path=FIGURES_DIR / "feature_combination_comparison.png",
)


## 3. 分类器对比实验

使用最佳特征组合，对比不同分类器的性能。


In [ ]:
# Use best feature combination (you can change this based on results)
best_feat_name = "HOG + Color + LBP"  # Update based on results
X_train_best = feature_combinations[best_feat_name]["train"]
X_val_best = feature_combinations[best_feat_name]["val"]
X_test_best = feature_combinations[best_feat_name]["test"]

print(f"Using feature combination: {best_feat_name}")
print(f"Feature dimension: {X_train_best.shape[1]}")

# Test different classifiers
classifier_results = []

# SVM
print("\nTraining SVM...")
svm_model = train_svm(X_train_best, y_train, kernel="rbf", C=1.0, class_weight="balanced")
svm_metrics = evaluate_model(svm_model, X_test_best, y_test, class_names, verbose=False)
classifier_results.append({
    "classifier": "SVM (RBF)",
    "accuracy": svm_metrics["accuracy"],
    "macro_f1": svm_metrics["macro_f1"],
    "weighted_f1": svm_metrics["weighted_f1"],
})

# Logistic Regression
print("\nTraining Logistic Regression...")
lr_model = train_logistic_regression(X_train_best, y_train, C=1.0, class_weight="balanced")
lr_metrics = evaluate_model(lr_model, X_test_best, y_test, class_names, verbose=False)
classifier_results.append({
    "classifier": "Logistic Regression",
    "accuracy": lr_metrics["accuracy"],
    "macro_f1": lr_metrics["macro_f1"],
    "weighted_f1": lr_metrics["weighted_f1"],
})

# MLP
print("\nTraining MLP...")
mlp_model = train_mlp(X_train_best, y_train, hidden_layer_sizes=(128, 64), max_iter=500)
mlp_metrics = evaluate_model(mlp_model, X_test_best, y_test, class_names, verbose=False)
classifier_results.append({
    "classifier": "MLP (128, 64)",
    "accuracy": mlp_metrics["accuracy"],
    "macro_f1": mlp_metrics["macro_f1"],
    "weighted_f1": mlp_metrics["weighted_f1"],
})

classifier_df = pd.DataFrame(classifier_results)
print("\n" + "="*60)
print("Classifier Comparison Results")
print("="*60)
print(classifier_df.to_string(index=False))


In [ ]:
# Visualize classifier comparison
plot_classifier_comparison(
    classifier_df["classifier"].tolist(),
    {
        "Accuracy": classifier_df["accuracy"].tolist(),
        "Macro F1": classifier_df["macro_f1"].tolist(),
    },
    metric_name="Accuracy",
    save_path=FIGURES_DIR / "classifier_comparison.png",
)


## 4. 保存所有结果

汇总所有消融实验结果。


In [ ]:
# Save results
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

feature_df.to_csv(LOGS_DIR / f"ablation_features_{timestamp}.csv", index=False)
classifier_df.to_csv(LOGS_DIR / f"ablation_classifiers_{timestamp}.csv", index=False)

print("Results saved to outputs/logs/")
print("\nSummary:")
print(f"Best feature combination: {feature_df.loc[feature_df['test_accuracy'].idxmax(), 'feature']}")
print(f"Best classifier: {classifier_df.loc[classifier_df['accuracy'].idxmax(), 'classifier']}")
